# Alto Rendimiento de Pandas: eval() y query()

Como ya hemos visto en las secciones anteriores, el poder de la pila PyData se basa en la capacidad de NumPy y Pandas para insertar operaciones básicas en C a través de una sintaxis intuitiva: los ejemplos son operaciones vectorizadas/transmitidas en NumPy y operaciones de tipo agrupación. en Pandas.
Si bien estas abstracciones son eficientes y efectivas para muchos casos de uso comunes, a menudo se basan en la creación de objetos intermedios temporales, lo que puede causar una sobrecarga indebida en el tiempo de cómputo y el uso de la memoria.
A partir de la versión 0.13 (lanzada en enero de 2014), Pandas incluye algunas herramientas experimentales que le permiten acceder directamente a operaciones de velocidad C sin la costosa asignación de arreglos intermedios.
Estas son las funciones ``eval()`` y ``query()``, que dependen del paquete [Numexpr](https://github.com/pydata/numexpr).
En este cuaderno, analizaremos su uso y daremos algunas reglas generales sobre cuándo podría pensar en usarlos.

## Motivando ``query()`` y ``eval()``: Expresiones compuestas
Hemos visto anteriormente que NumPy y Pandas admiten operaciones vectorizadas rápidas; por ejemplo, al sumar los elementos de dos arreglos:

In [ ]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

100 loops, best of 3: 3.39 ms per loop


Como se discutió en [Computación en Arrays NumPy: Funciones universales](02.03-Computacion-en-arrays-ufuncs.ipynb), esto es mucho más rápido que hacer la adición a través de un bucle o comprensión de Python:

In [ ]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

1 loop, best of 3: 266 ms per loop


Pero esta abstracción puede volverse menos eficiente cuando se calculan expresiones compuestas.
Por ejemplo, considere la siguiente expresión:

In [ ]:
mask = (x > 0.5) & (y < 0.5)

Debido a que NumPy evalúa cada subexpresión, esto equivale aproximadamente a lo siguiente:

In [ ]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

En otras palabras, *cada paso intermedio se asigna explícitamente en la memoria*. Si las matrices ``x`` e ``y`` son muy grandes, esto puede generar una sobrecarga de memoria y computacional significativa.
La biblioteca Numexpr le brinda la capacidad de calcular este tipo de expresión compuesta elemento por elemento, sin necesidad de asignar matrices intermedias completas.
La [Numexpr documentation](https://github.com/pydata/numexpr) tiene más detalles, pero por el momento es suficiente decir que la biblioteca acepta una *cadena* que proporciona la expresión de estilo NumPy que le gustaría calcular:

In [ ]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

El beneficio aquí es que Numexpr evalúa la expresión de una manera que no usa arreglos temporales de tamaño completo y, por lo tanto, puede ser mucho más eficiente que NumPy, especialmente para arreglos grandes.
Las herramientas de Pandas ``eval()`` y ``query()`` que discutiremos aquí son conceptualmente similares y dependen del paquete Numexpr.

## ``pandas.eval()`` para operaciones eficientes
La función ``eval()`` en Pandas usa expresiones de cadena para calcular operaciones de manera eficiente usando ``DataFrame``s.
Por ejemplo, considere los siguientes ``DataFrame``s:

In [ ]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

Para calcular la suma de los cuatro ``DataFrame``s usando el enfoque típico de Pandas, podemos simplemente escribir la suma:

In [ ]:
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 87.1 ms per loop


El mismo resultado se puede calcular a través de ``pd.eval`` construyendo la expresión como una cadena:

In [ ]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 42.2 ms per loop


La versión ``eval()`` de esta expresión es aproximadamente un 50% más rápida (y usa mucha menos memoria), mientras que da el mismo resultado:

In [ ]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

### Operaciones soportadas por ``pd.eval()``
A partir de Pandas v0.16, ``pd.eval()`` admite una amplia gama de operaciones.
Para demostrar esto, usaremos el siguiente entero ``DataFrame``s:

In [ ]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### Operadores aritméticos
``pd.eval()`` admite todos los operadores aritméticos. Por ejemplo:

In [ ]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Operadores de comparación
``pd.eval()`` admite todos los operadores de comparación, incluidas las expresiones encadenadas:

In [ ]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Operadores bit a bit
``pd.eval()`` admite los operadores bit a bit ``&`` y ``|``:

In [ ]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

Además, admite el uso de los literales ``y`` y ``o`` en expresiones booleanas:

In [ ]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Atributos e índices de objetos
``pd.eval()`` admite el acceso a los atributos de los objetos mediante la sintaxis ``obj.attr`` y los índices mediante la sintaxis ``obj[index]``:

In [ ]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Otras operaciones
Otras operaciones tales como llamadas a funciones, declaraciones condicionales, bucles y otras construcciones más involucradas actualmente *no* están implementadas en ``pd.eval()``.
Si desea ejecutar estos tipos de expresiones más complicados, puede usar la propia biblioteca Numexpr.

## ``DataFrame.eval()`` para operaciones de columnas
Así como Pandas tiene una función ``pd.eval()`` de nivel superior, ``DataFrame``s tiene un método ``eval()`` que funciona de manera similar.
El beneficio del método ``eval()`` es que se puede hacer referencia a las columnas *por su nombre*.
Usaremos esta matriz etiquetada como ejemplo:

In [ ]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Usando ``pd.eval()`` como arriba, podemos calcular expresiones con las tres columnas como esta:

In [ ]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

El método ``DataFrame.eval()`` permite una evaluación mucho más sucinta de expresiones con las columnas:

In [ ]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

Observe aquí que tratamos los *nombres de columna como variables* dentro de la expresión evaluada, y el resultado es el que desearíamos.

### Asignación en DataFrame.eval()
Además de las opciones recién discutidas, ``DataFrame.eval()`` también permite la asignación a cualquier columna.
Usemos el ``DataFrame`` de antes, que tiene las columnas ``'A'``, ``'B'`` y ``'C'``:

In [ ]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


Podemos usar ``df.eval()`` para crear una nueva columna ``'D'`` y asignarle un valor calculado a partir de las otras columnas:

In [ ]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


De la misma forma se puede modificar cualquier columna existente:

In [ ]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### Variables locales en DataFrame.eval()
El método ``DataFrame.eval()`` admite una sintaxis adicional que le permite trabajar con variables locales de Python.
Considera lo siguiente:

In [ ]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

El carácter ``@`` aquí marca un *nombre de variable* en lugar de un *nombre de columna*, y le permite evaluar de manera eficiente las expresiones que involucran los dos "espacios de nombres": el espacio de nombres de las columnas y el espacio de nombres de los objetos de Python.
Tenga en cuenta que este carácter ``@`` solo es compatible con el *método* ``DataFrame.eval()``, no con la *función* ``pandas.eval()``, porque ``pandas.eval ()`` solo tiene acceso al espacio de nombres (Python).

## Método DataFrame.query()
El ``DataFrame`` tiene otro método basado en cadenas evaluadas, llamado método ``query()``.
Considera lo siguiente:

In [ ]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

Como en el ejemplo usado en nuestra discusión de ``DataFrame.eval()``, esta es una expresión que involucra columnas de ``DataFrame``.
¡Sin embargo, no se puede expresar utilizando la sintaxis ``DataFrame.eval()``!
En cambio, para este tipo de operación de filtrado, puedes usar el método ``query()``:

In [ ]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

Además de ser un cálculo más eficiente, en comparación con la expresión de enmascaramiento, es mucho más fácil de leer y comprender.
Tenga en cuenta que el método ``query()`` también acepta el indicador ``@`` para marcar variables locales:

In [ ]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Rendimiento: cuándo usar estas funciones
Al considerar si usar estas funciones, hay dos consideraciones: *tiempo de cómputo* y *uso de memoria*.
El uso de la memoria es el aspecto más predecible. Como ya se mencionó, cada expresión compuesta que involucre arreglos NumPy o Pandas ``DataFrame``s resultará en la creación implícita de arreglos temporales:
Por ejemplo, esto:

In [ ]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

Es más o menos equivalente a esto:

In [ ]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

Si el tamaño de los ``DataFrame``s temporales es significativo en comparación con la memoria disponible del sistema (típicamente varios gigabytes), entonces es una buena idea usar una expresión ``eval()`` o ``query()``.
Puede verificar el tamaño aproximado de su matriz en bytes usando esto:

In [ ]:
df.values.nbytes

32000

Por el lado del rendimiento, ``eval()`` puede ser más rápido incluso cuando no está maximizando la memoria de su sistema.
El problema es cómo se comparan sus ``DataFrame`` temporales con el tamaño de la memoria caché de la CPU L1 o L2 en su sistema (normalmente unos pocos megabytes en 2016); si son mucho más grandes, entonces ``eval()`` puede evitar algunos movimientos potencialmente lentos de valores entre los diferentes cachés de memoria.
En la práctica, encuentro que la diferencia en el tiempo de cálculo entre los métodos tradicionales y el método ``eval``/``query`` generalmente no es significativa; en todo caso, ¡el método tradicional es más rápido para arreglos más pequeños!
El beneficio de ``eval``/``query`` está principalmente en la memoria guardada y en la sintaxis a veces más limpia que ofrecen.
Hemos cubierto la mayoría de los detalles de ``eval()`` y ``query()`` aquí; para obtener más información sobre estos, puede consultar la documentación de Pandas.
En particular, se pueden especificar diferentes analizadores y motores para ejecutar estas consultas; para obtener detalles sobre esto, consulte la discusión dentro de la sección ["Enhancing Performance"](http://pandas.pydata.org/pandas-docs/dev/enhancingperf.html).